In [3]:
!pip install requests
!pip install beautifulsoup4

In [51]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd
import re

In [52]:
tourn_ids=["2039","1944","1848","1797","1710",
          "2040","1959","1849","1798","1711",
          "2018","1960","1850","1799","1712",
          "2019","1961","1851","1800","1713"]

In [53]:
def get_teams(tourn_id):
    url = "https://www.soccerbase.com/tournaments/tournament.sd?tourn_id="+tourn_id
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    stats_elements = soup.find_all("td",class_="bull")
    teams= []
    for i in stats_elements:
        teams.append(i.text)
    return teams

In [54]:
def get_ids(tourn_id):
    url = "https://www.soccerbase.com/tournaments/tournament.sd?tourn_id="+tourn_id
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    stats_elements = soup.find_all("td",class_="bull")
    ids = []
    for i in stats_elements:
        ids.append(i.find('a')['href'].split('=')[1])
    return ids

In [55]:
all_teams = []
for tourn_id in tourn_ids:
    for i in get_teams(tourn_id):
        all_teams.append(i)

In [56]:
all_ids = []
for tourn_id in tourn_ids:
    for i in get_ids(tourn_id):
        all_ids.append(i)

In [57]:
all_teams = list(set(all_teams))

In [58]:
all_ids = list(set(all_ids))

In [59]:
def get_players(team_id):
    player_names=[]
    url = "https://www.soccerbase.com/teams/team.sd?team_id="+team_id+"&teamTabs=details"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    stats_elements = soup.find_all("li",class_="alt")
    for i in stats_elements:
        if i.find('a'):
            player_names.append(i.find('a').text)
    return player_names

In [60]:
url = "https://www.soccerbase.com/teams/team.sd?team_id=652&teamTabs=details"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
stats_elements = soup.find_all("li",class_="alt")
for i in stats_elements:
    if i.find('a'):
        print(i.find('a')['href'].split('=')[1])

163910
186008
139146
114413
159502
183219
186009
99958
109180
55832
138067
165243
74287
127043
73572
140587
90083
182804
175860
178183


In [61]:
all_players = []
for team_id in all_ids:
    players = get_players(team_id)
    for i in players:
        all_players.append(i)

In [62]:
all_players = list(set(all_players))

In [63]:
def get_player_ids(team_id):
    player_ids=[]
    url = "https://www.soccerbase.com/teams/team.sd?team_id="+team_id+"&teamTabs=details"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    stats_elements = soup.find_all("li",class_="alt")
    for i in stats_elements:
        if i.find('a'):
            player_ids.append(i.find('a')['href'].split('=')[1])
    return player_ids

In [64]:
all_player_ids = []
for team_id in all_ids:
    player_ids = get_player_ids(team_id)
    for i in player_ids:
        all_player_ids.append(i)

In [65]:
all_player_ids = list(set(all_player_ids))

In [66]:
def extract_except_date(input_string):
    # Define the pattern for the date in the format '%b %y'
    date_pattern = r'\b\d{2}\s+(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\b,'
    
    # Find all occurrences of the date pattern in the string
    dates = re.findall(date_pattern, input_string)
    
    # Replace the dates found in the string with an empty string
    for date in dates:
        input_string = input_string.replace(date, '')
    
    return input_string.strip()

In [79]:
def get_player_career(player_id):
    career = []
    url = "https://www.soccerbase.com/players/player.sd?player_id="+player_id
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    stats_elements = soup.findAll('tbody')[0].findAll('tr')
    for i in stats_elements[:-2]:
        if 'Youth' not in i.text.split(' ') and 'Trainee' not in i.text.split(' '):
            team = i.text.split(re.search(r'\d+', i.text.split(', ')[0]).group())[0][:-1]
            if team == '':
                team = extract_except_date(i.text.split(re.search(r'\d+', i.text.split(', ')[1]).group())[0][:-1])
            if len(i.text.split(', ')) == 2:
                from_ = datetime.strptime(' '.join([i.text.split(', ')[0][-3:],i.text.split(', ')[1][:2]]),'%b %y')
                # Get the current month and year
                current_month = datetime.now().month
                current_year = datetime.now().year
                # Create a datetime object for the first day of the current month
                to_ = datetime(current_year, current_month, 1, 0, 0)
            else:
                from_ = datetime.strptime(' '.join([i.text.split(', ')[0][-3:],i.text.split(', ')[1][:2]]),'%b %y')
                to_ =  datetime.strptime(' '.join([i.text.split(', ')[1][-3:],i.text.split(', ')[2][:2]]),'%b %y')
            career.append([team,from_,to_])
    return career 

In [85]:
get_player_career('134157')

[['Walsall',
  datetime.datetime(2023, 7, 1, 0, 0),
  datetime.datetime(2024, 2, 1, 0, 0)],
 ['Walsall',
  datetime.datetime(2023, 1, 1, 0, 0),
  datetime.datetime(2023, 5, 1, 0, 0)],
 ['Kettering',
  datetime.datetime(2021, 9, 1, 0, 0),
  datetime.datetime(2022, 6, 1, 0, 0)],
 ['Wolves',
  datetime.datetime(2020, 7, 1, 0, 0),
  datetime.datetime(2023, 7, 1, 0, 0)]]

In [69]:
def insert_space_before_capitals_except_first(input_string):
    result = [input_string[0]]  # Initialize result with the first character
    for char in input_string[1:]:
        if char.isupper():
            result.append(' ')  # Insert a space before each capital letter
        result.append(char)
    return ''.join(result)

# Example usage:
input_string = 'OmarBogleIsGreat'

result = insert_space_before_capitals_except_first(input_string)
print("Modified string:", result)

Modified string: Omar Bogle Is Great


In [81]:
def get_player_name(player_id):
    url = "https://www.soccerbase.com/players/player.sd?player_id="+player_id
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    stats_elements = soup.find_all("table",class_="imageHead")
    input_string = stats_elements[0].find('tr').text
    return insert_space_before_capitals_except_first(''.join(re.findall(r'[a-zA-Z]', input_string)))

In [82]:
get_player_name('53707')

'Thiago'

In [83]:
get_player_name('79428')

'Omar Bogle'

In [86]:
data = []
for player_id in all_player_ids:
    print(get_player_name(player_id))
    name = get_player_name(player_id)
    career = get_player_career(player_id)
    for i in career:
        data.append([player_id, name, i[0],i[1],i[2]])

Micah Mbick
Aaron Morley
George Edmundson
Jordan Slew
Toby Sims
Joel Matip
Sam Hutchinson
Jason Steele
Alhagi Sisay
Max Melbourne
Max Edgecombe
Josh Koroma
Alex Borto
John Swift
Kyle Cameron
Jordan Roberts
Antoine Semenyo
Josef Yarney
Ethan Laird
Joe Lumley
Liam Millar
Pervis Estupinan
Oli Mc Burnie
Alex Mac Donald
Emmanuel Onariase
Matthew Hudson
Myles Cowling
Chris Maxwell
James Webster
Christ Tiehi
Elkan Baggott
Tom Bradbury
Diamond Edwards
Lewis Brunt
Kelvin Abrefa
Noni Madueke
Tom Lockyer
Neo Dobson
Zeki Fryers
Finley Munroe
Lee Angol
Lucas Sant
Otis Khan
Slobodan Tedic
Tavonga Kuleya
Taylan Harris
Luther Munakandafa
Leon Ayinde
Josh Ruffles
Grant Hanley
Reece Brown
Kelvin Mellor
Patrick Gamble
Bobby Faulkner
Dan Gosling
Jack Wells Morrison
Darnell Furlong
Lewis Fensome
Leo Hjelde
Charlie Winfield
Johnson Opawole
Jamie Walker
Michael Olise
Daniel Mandroiu
Zak Baker
Jack Grimmer
Matty Palmer
Christian Saydee
Nick Tsaroulla
Sam Long
Josh Hallard
Jonathan Tomkinson
Danny Edwards
Eogh

Adebayo Fapetu
Kieran Green
Ethan Brierley
Ryan Rydel
Jesse Aldridge
Paris Lock
Tom Smith
Ronan Curtis
Joseph Snowdon
Cole Stockton
Illia Zabarnyi
Matty Cash
Archie Cunningham
Emiliano Buendia
Dimitrios Goutas
David Ozoh
Rico Richards
Kyran Lofthouse
Diogo Dalot
Matt Stevens
Ryan Sandford
Anjola Popoola
Kieran Sadlier
Jacob Mendy
Harry O Gara
Dan Barton
Sean Holland
Jake Kean
Haydon Roberts
J J Mc Kiernan
Jack Powell
Ollie Younger
Charlie Austin
Loris Karius
Matt Macey
James Brophy
Bastian Smith
Jamari Hart
Mac Kenzie Warne
Issa Diop
Scott Loach
Kelechi Etienne
Jason Demetriou
Gavin Massey
Josh Ayres
Jack Currie
Connor Ogilvie
Promise Omochere
Shadrach Ogie
Bambo Diaby
William Alves
Korey Smith
Jack Moylan
Brooke Norton Cuffy
Darien Wauchope
Issa Kabore
Josh Sheehan
Nico O Reilly
Zach Mitchell
Ben Heneghan
Liam Sercombe
Harry Jewitt White
Dom Telford
Shane Ferguson
Isaac Burgess
Conor Thomas
Dermi Lusala
Tom Bloxham
Matthew Smith
Aaron Ramsey
Cameron Mc Geehan
Matt Godden
Jeffrey Forku

Aidan Keena
Matt Kerbey
Jeremy Ngakia
Matt Butcher
Sam Gale
James Ward Prowse
Aidan Elliott Wheeler
Michael Ihiekwe
Harry Hawkins
Gui Siqueira
Murphy Bennett
Mikolaj Lenarcik
Ben Purrington
Zach Marsh
Andy Cannon
Nathan Ferguson
Shane Mc Loughlin
Arjan Raikhy
Archie Collins
Oliver Tipton
Panutche Camara
Bernd Leno
Will Randall
Edward James
Brennan Johnson
Jacob Maddox
Sam Clucas
Matthew Cox
George Hirst
Jonathan Leko
William Lakin
Bradley Williams
Clinton Nosakhare
Christian Norgaard
William Hondermarck
Josh Davison
Ibou Touray
James Dadge
Mikel Miller
Joe Hodge
Jack Taylor
Tobi Adebayo Rowling
Thenisoklis Kafalas
Aden Flint
George Friend
Jack Shepherd
Ederson Moraes
Haji Wright
Vadaine Oliver
Jordan Smith
James Wilson
Ty Webster
Josiah Dyer
Danny Johnson
Bae Seo Joon
Ethan Ebanks Landell
Olamide Shodipo
Ty Barnett
Craig Dawson
Kofi Balmer
Conor Falls
Ted Curd
Jonny Smith
Kristian Dennis
Kayden Jackson
Jackson Smith
Gavan Holohan
Jacob Butterfield
Freddie Willcox
Connor Taylor
Nickseon

Harry Wilson
Daniel Happe
Joe Riley
Joe Hugill
Ryley Towler
Tyler Adams
Christian Walton
Vimal Yoganathan
Benjamin Fredrick
Jamie Sterry
Adedeji Oshilaja
Joe Grey
Warren O Hora
Bryan Gil
Massimo Luongo
John Ruddy
Harvey Tomlinson
Willy Boly
Callum Marshall
Luke Thomas
Max Lowe
Charlie Pegrum
Morgan Whittaker
Alexander Isak
Sean Clare
Aaron Bott
John Marquis
Teddy Mfuni
Louis Chadwick
Josh Dockerill
Jacob Murphy
Sam Waller
Ben Wilson
John Mc Atee
Lewis Cook
Owen Lunt
Mark O Mahony
Aaron Atkinson
Simba Nyamwanza
Lewis Leigh
Jaydan Davidson
Odin Bailey
Neto
Tom Bayliss
Lamine Toure
Dominic Calvert Lewin
Oliver Riva
Joe Mattock
Divock Origi
Oliver Finney
Will Keane
Rio Cragg
Nathan Redmond
Paris Maghoma
Isaac Fletcher
Corey Edwards
Luc De Fougerolles
Connor Mahoney
James Henry


In [87]:
df = pd.DataFrame(data, columns=['ID', 'Name', 'Team', 'From', 'To'])

In [88]:
df.head(25)

,ID,Name,Team,From,To
0,185807,Micah Mbick,Charlton,2023-11-01,2024-02-01
1,94489,Aaron Morley,Bolton,2022-01-01,2024-02-01
2,94489,Aaron Morley,Rochdale,2016-08-01,2022-01-01
3,86915,George Edmundson,Ipswich,2021-07-01,2024-02-01
4,86915,George Edmundson,Derby,2021-02-01,2021-06-01
5,86915,George Edmundson,Rangers,2019-06-01,2021-07-01
6,86915,George Edmundson,Fylde,2017-10-01,2018-01-01
7,86915,George Edmundson,Alfreton,2016-09-01,2017-01-01
8,86915,George Edmundson,Oldham,2015-07-01,2019-06-01
9,57203,Jordan Slew,Morecambe,2023-07-01,2024-02-01


In [90]:
df.to_csv('player_data.csv', index=False)